In [1]:
import pandas as pd
import numpy as np

rawData_org = pd.read_csv('./NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [2]:
rawData_org.head()

,기준년도,가입자일련번호,성별코드,연령대코드(5세단위),시도코드,신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),...,감마지티피,흡연상태,음주여부,구강검진 수검여부,치아우식증유무,결손치유무,치아마모증유무,제3대구치(사랑니)이상,치석,데이터공개일자
0,2017,1,1,13,46,170.0,65.0,91.0,1.0,1.2,...,25.0,3.0,0.0,1,NaN,NaN,NaN,NaN,1.0,20181126
1,2017,2,2,8,41,150.0,45.0,73.4,1.2,1.0,...,10.0,1.0,0.0,1,NaN,NaN,NaN,NaN,1.0,20181126
2,2017,3,1,8,45,175.0,75.0,94.0,1.0,0.8,...,136.0,1.0,0.0,1,NaN,NaN,NaN,NaN,0.0,20181126
3,2017,4,2,12,11,155.0,55.0,67.5,0.9,1.0,...,30.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN,20181126
4,2017,5,1,8,41,175.0,75.0,93.0,1.5,1.5,...,68.0,3.0,0.0,0,NaN,NaN,NaN,NaN,NaN,20181126


In [3]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [4]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [5]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [24]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx[:10]

[4, 34, 46, 56, 61, 62, 85, 127, 172, 182]

In [25]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정


# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,1,8,175.0,75.0,93.0,1.5,1.5,1.0,1.0,128.0,...,182.0,47.0,169.0,1.0,1.1,37.0,62.0,68.0,3.0,0.0
1,2,5,155.0,40.0,70.0,1.5,1.5,1.0,1.0,129.0,...,110.0,56.0,92.0,1.0,0.5,19.0,14.0,31.0,2.0,1.0
2,2,10,155.0,50.0,70.0,1.2,1.2,1.0,1.0,115.0,...,69.0,50.0,144.0,1.0,0.9,18.0,13.0,15.0,1.0,0.0
3,2,6,155.0,65.0,74.0,0.3,0.3,1.0,1.0,99.0,...,56.0,61.0,57.0,1.0,0.6,19.0,5.0,10.0,1.0,1.0
4,1,12,165.0,65.0,87.0,1.0,0.8,1.0,1.0,124.0,...,121.0,43.0,100.0,1.0,0.8,26.0,25.0,17.0,3.0,0.0
5,2,5,165.0,55.0,80.0,0.9,0.9,1.0,1.0,116.0,...,26.0,54.0,64.0,1.0,0.7,10.0,10.0,10.0,1.0,1.0
6,1,12,180.0,70.0,83.0,1.0,0.5,1.0,1.0,119.0,...,108.0,60.0,122.0,1.0,1.0,17.0,11.0,25.0,3.0,1.0
7,1,9,180.0,85.0,93.0,0.9,1.2,1.0,1.0,132.0,...,65.0,41.0,84.0,1.0,0.7,20.0,19.0,26.0,3.0,1.0
8,1,7,170.0,65.0,76.1,0.5,0.7,1.0,1.0,102.0,...,50.0,54.0,83.0,1.0,1.0,25.0,20.0,17.0,1.0,1.0
9,2,10,160.0,60.0,81.0,0.6,0.5,1.0,1.0,111.0,...,79.0,53.0,81.0,1.0,0.6,18.0,14.0,10.0,1.0,1.0


In [26]:
rawData_sample.shape

(50000, 22)

In [37]:
feature_columns_to_use = ['성별코드', '연령대코드', '신장', '체중', '허리둘레', 
                          '시력_좌', '시력_우', '청력_좌', '청력_우', '수축기혈압', 
                          '이완기혈압', '식전혈당', '트리글리세라이드', 'HDL콜레스테롤','LDL콜레스테롤',
                          '요단백', '혈청크레아티닌', '혈청지오티_AST', '혈청지오티_ALT', '감마지티피', 
                          '흡연상태', '음주여부']

rawData_sample.rename(columns = {'연령대코드(5세단위)' : '연령대코드',
                                 '신장(5Cm단위)' : '신장',
                                 '체중(5Kg 단위)' : '체중',
                                 '시력(좌)' : '시력_좌',
                                 '시력(우)' : '시력_우',
                                 '청력(좌)' : '청력_좌',
                                 '청력(우)' : '청력_우',
                                 '식전혈당(공복혈당)' : '식전혈당',
                                 '(혈청지오티)AST' : '혈청지오티_AST',
                                 '(혈청지오티)ALT' : '혈청지오티_ALT'}, inplace = True)

# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])
    
rawData_sample .head()

,성별코드,연령대코드,신장,체중,허리둘레,시력_좌,시력_우,청력_좌,청력_우,수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,혈청지오티_AST,혈청지오티_ALT,감마지티피,흡연상태,음주여부
0,0,8,175.0,75.0,93.0,1.5,1.5,1.0,1.0,128.0,...,182.0,47.0,169.0,1.0,1.1,37.0,62.0,68.0,3.0,0
1,1,5,155.0,40.0,70.0,1.5,1.5,1.0,1.0,129.0,...,110.0,56.0,92.0,1.0,0.5,19.0,14.0,31.0,2.0,1
2,1,10,155.0,50.0,70.0,1.2,1.2,1.0,1.0,115.0,...,69.0,50.0,144.0,1.0,0.9,18.0,13.0,15.0,1.0,0
3,1,6,155.0,65.0,74.0,0.3,0.3,1.0,1.0,99.0,...,56.0,61.0,57.0,1.0,0.6,19.0,5.0,10.0,1.0,1
4,0,12,165.0,65.0,87.0,1.0,0.8,1.0,1.0,124.0,...,121.0,43.0,100.0,1.0,0.8,26.0,25.0,17.0,3.0,0


In [38]:
feature_columns_to_use[0:len(feature_columns_to_use)] 

['성별코드',
 '연령대코드',
 '신장',
 '체중',
 '허리둘레',
 '시력_좌',
 '시력_우',
 '청력_좌',
 '청력_우',
 '수축기혈압',
 '이완기혈압',
 '식전혈당',
 '트리글리세라이드',
 'HDL콜레스테롤',
 'LDL콜레스테롤',
 '요단백',
 '혈청크레아티닌',
 '혈청지오티_AST',
 '혈청지오티_ALT',
 '감마지티피',
 '흡연상태',
 '음주여부']

In [39]:
# 소스 데이터프레임에서 예측(prediction)을 위한 속성 집합
# X = rawData.loc[:, feature_columns_to_use[0:8, 10:21]]  
y = rawData_sample.loc[:, feature_columns_to_use[9]]  # 종속변수(예측속성, 수축기혈압)
del feature_columns_to_use[9]
X = rawData_sample.loc[:, feature_columns_to_use]  # 독립변수

In [40]:
X.head()

,성별코드,연령대코드,신장,체중,허리둘레,시력_좌,시력_우,청력_좌,청력_우,이완기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,혈청지오티_AST,혈청지오티_ALT,감마지티피,흡연상태,음주여부
0,0,8,175.0,75.0,93.0,1.5,1.5,1.0,1.0,80.0,...,182.0,47.0,169.0,1.0,1.1,37.0,62.0,68.0,3.0,0
1,1,5,155.0,40.0,70.0,1.5,1.5,1.0,1.0,84.0,...,110.0,56.0,92.0,1.0,0.5,19.0,14.0,31.0,2.0,1
2,1,10,155.0,50.0,70.0,1.2,1.2,1.0,1.0,70.0,...,69.0,50.0,144.0,1.0,0.9,18.0,13.0,15.0,1.0,0
3,1,6,155.0,65.0,74.0,0.3,0.3,1.0,1.0,75.0,...,56.0,61.0,57.0,1.0,0.6,19.0,5.0,10.0,1.0,1
4,0,12,165.0,65.0,87.0,1.0,0.8,1.0,1.0,82.0,...,121.0,43.0,100.0,1.0,0.8,26.0,25.0,17.0,3.0,0


In [41]:
y.head()

0    128.0
1    129.0
2    115.0
3     99.0
4    124.0
Name: 수축기혈압, dtype: float64

In [42]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [43]:
features = ""

for attr in feature_columns_to_use[:-1] :
    features = features + attr + "+"

features = features + feature_columns_to_use[len(feature_columns_to_use)-1]    
features

'성별코드+연령대코드+신장+체중+허리둘레+시력_좌+시력_우+청력_좌+청력_우+이완기혈압+식전혈당+트리글리세라이드+HDL콜레스테롤+LDL콜레스테롤+요단백+혈청크레아티닌+혈청지오티_AST+혈청지오티_ALT+감마지티피+흡연상태+음주여부'

In [44]:
# Break into left and right hand side; y and x
y, X = dmatrices('수축기혈압~'+features, data=rawData_sample, return_type="dataframe")

In [45]:
# For each Xi, calculate VIF
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Fit X to y
result = sm.OLS(y, X).fit()

In [46]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  수축기혈압   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     3584.
Date:                Fri, 12 Jul 2019   Prob (F-statistic):               0.00
Time:                        11:04:48   Log-Likelihood:            -1.8189e+05
No. Observations:               50000   AIC:                         3.638e+05
Df Residuals:                   49978   BIC:                         3.640e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     42.3928      1.473     28.780      0.000      39.506      45.280
성별코드          -1.1861      0.144     -8.254      0.000      -1.468      -0.904
연령대코드          0.8673      0.018     47.151      0.000       0.831       0.903
신장            -0.1083      0.008    -13.406      0.000      -0.124      -0.092
체중             0.1351      0.006     23.989      0.000       0.124       0.146
허리둘레           0.0194      0.004      4.886      0.000       0.012       0.027
시력_좌          -0.1319      0.077     -1.719      0.086      -0.282       0.019
시력_우          -0.1541      0.073     -2.104      0.035      -0.298      -0.011
청력_좌           1.3918      0.278      5.002      0.000       0.846       1.937
청력_우           0.0844      0.284      0.297      0.767      -0.473       0.641
이완기혈압          1.0037      0.004    224.999      0.000       0.995       1.012
식전혈당           0.0194      0.002     10.798      0.000       0.016       0.023
트리글리세라이드       0.0008      0.000      1.622      0.105      -0.000       0.002
HDL콜레스테롤       0.0062      0.003      1.975      0.048    4.81e-05       0.012
LDL콜레스테롤      -0.0090      0.001     -7.811      0.000      -0.011      -0.007
요단백            0.5015      0.093      5.389      0.000       0.319       0.684
혈청크레아티닌        0.0596      0.077      0.775      0.438      -0.091       0.210
혈청지오티_AST      0.0071      0.003      2.029      0.042       0.000       0.014
혈청지오티_ALT     -0.0077      0.003     -2.607      0.009      -0.014      -0.002
감마지티피          0.0020      0.001      2.080      0.038       0.000       0.004
흡연상태          -0.3659      0.064     -5.744      0.000      -0.491      -0.241
음주여부          -0.2287      0.096     -2.389      0.017      -0.416      -0.041
==============================================================================
Omnibus:                     2999.608   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5087.730
Skew:                           0.476   Prob(JB):                         0.00
Kurtosis:                       4.239   Cond. No.                     1.09e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [47]:
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [48]:
vif.round(1)

,VIF Factor,features
0,1282.1,Intercept
1,3.0,성별코드
2,1.6,연령대코드
3,3.4,신장
4,3.1,체중
5,1.7,허리둘레
6,1.2,시력_좌
7,1.1,시력_우
8,1.4,청력_좌
9,1.4,청력_우


In [49]:
# RMSE
float(np.mean((result.predict(X) - rawData_sample['수축기혈압'])**2))

84.58032967391273

In [50]:
# RMSLE 계산하는 사용자정의 함수
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values) :
    # 넘파이로 배열 형태로 변환
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제값에 1을 더하고 로그를 씌움
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 함
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균값 구함
    mean_difference = difference.mean()
    
    # 다시 루트를 씌움
    score = np.sqrt(mean_difference)
    
    return score

#rmsle_scorer = make_scorer(rmsle)
#rmsle_scorer

rmsle_score = rmsle(result.predict(X), rawData_sample['수축기혈압'])
rmsle_score

0.07401461592118207

In [51]:
(result.predict(X) - rawData_sample['수축기혈압']).tail(100)

49900   -17.887380
49901   -13.130186
49902     9.013389
49903   -14.482023
49904    -4.123932
49905     9.793507
49906   -12.243782
49907     0.566928
49908   -14.213660
49909     0.829560
49910     3.252070
49911    -2.552367
49912     1.636557
49913     3.344081
49914     6.374236
49915    -0.863588
49916    -8.809002
49917     6.970725
49918     0.932631
49919     4.896446
49920     3.471956
49921    -8.324826
49922     5.959237
49923    -1.448959
49924     0.125503
49925     6.164171
49926     9.225915
49927     3.638610
49928     3.419222
49929    -7.605162
           ...    
49970     5.700061
49971    12.240728
49972    -3.701476
49973    -1.741417
49974    -5.443499
49975    -7.837714
49976   -19.388440
49977    -2.508585
49978     3.433514
49979     5.078587
49980    -7.738941
49981    -2.978034
49982    -9.351809
49983    -8.022748
49984    15.927320
49985   -38.509966
49986    15.811089
49987    -4.098219
49988     0.742977
49989    -2.709802
49990    11.502006
49991    -5.